In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score

In [16]:
df = pd.read_csv("./data/titanic.csv")
df.drop(columns=["sex", "deck", "class", "adult_male", "embark_town", "alive"], inplace=True)
df.dropna(inplace=True)

## TEAM CHALLENGE: TOOLBOX (II)

### Funcion: get_features_cat_classification



Esta función recibe como argumentos un dataframe, el nombre de una de las columnas del mismo (argumento 'target_col'), que debería ser el target de un hipotético modelo de clasificación, es decir debe ser una variable categórica o numérica discreta pero con baja cardinalidad, un argumento "normalize" con valor False por defecto, una variable float "mi_threshold" cuyo valor por defecto será 0.

* En caso de que "normalize" sea False:
    La función debe devolver una lista con las columnas categóricas del dataframe cuyo valor de mutual information con 'target_col' iguale o supere el valor de "mi_threshold".

* En caso de que "normalize" sea True:
    La función debe devolver una lista con las columnas categóricas del dataframe cuyo valor normalizado de mutual information con 'target_col' iguale o supere el valor de "mi_threshold". El valor normalizado de mutual information se considera el obtenido de dividir el valor de mutual information tal cual ofrece sklearn o la fórmula de cálculo por la suma de todos los valores de mutual information de las features categóricas del dataframe.
    En este caso, la función debe comprobar que "mi_threshold" es un valor float entre 0 y 1, y arrojar un error si no lo es.

La función debe hacer todas las comprobaciones necesarias para no dar error como consecuecia de los valores de entrada. Es decir hará un check de los valores asignados a los argumentos de entrada y si estos no son adecuados debe retornar None y printar por pantalla la razón de este comportamiento. Ojo entre las comprobaciones debe estar que "target_col" hace referencia a una variable categórica del dataframe.

In [21]:
def get_features_cat_classification(df, target_col = "", normalize = False, mi_threshold = 0):
    
    # Comprobación de errores de entrada
    if not isinstance(df, pd.DataFrame):
        print("El argumento 'df' debe ser un DataFrame de pandas.")
        return None
    if target_col not in df.columns:
        print("El argumento 'target_col' debe ser una columna válida en el DataFrame.")
        return None
    if not isinstance(mi_threshold, (int, float)):
        print("El argumento 'mi_threshold' debe ser un número.")
        return None    
    if not isinstance(normalize, bool):
        print("El argumento 'normalize' debe ser un valor booleano.")
        return None
    if df[target_col].nunique()/len(df) > .2:
        print(f"{target_col} no parece una columna categórica")
        return None
    
    if not normalize:
        mutual_info = mutual_info_score
    else:
        mutual_info = normalized_mutual_info_score
        if not 0 <= mi_threshold <= 1:
            print("Cuando 'normalize' es True, 'mi_threshold' debe estar entre 0 y 1.")
            return None
        
    columns = [col for col in df.columns if df[col].nunique() < 20 and col != target_col]
    para_pintar = []
    for col in columns:
        mis = mutual_info(df[target_col], df[col])
        if mis >= mi_threshold:
            para_pintar.append(col)
    return para_pintar

In [22]:
get_features_cat_classification(df, "survived")

['pclass', 'sibsp', 'parch', 'embarked', 'who', 'alone']

### Funcion: plot_features_cat_classification

Esta función recibe un dataframe, una argumento "target_col" con valor por defecto "", una lista de strings ("columns") cuyo valor por defecto es la lista vacía, un argumento ("mi_threshold") con valor 0.0 por defecto, y un argumento "normalize" a False.

Si la lista no está vacía:
* La función seleccionara de esta lista los valores que correspondan a columnas o features categóricas del dataframe cuyo valor de mutual information respecto de target_col supere el umbral puesto en "mi_threshold" (con las mismas considereciones respecto a "normalize" que se comentan en la descripción de la función "get_features_cat_classification").
* Para los valores seleccionados, pintará la distribución de etiquetas de cada valor respecto a los valores de la columna "target_col".

Si la lista está vacía:
* Entonces la función igualará "columns" a las variables categóricas del dataframe y se comportará como se describe en la sección "Si la lista no está vacía"

De igual manera que en la función descrita anteriormente deberá hacer un check de los valores de entrada y comportarse como se describe en el último párrafo de la función `get_features_cat_classification`.

In [13]:
def plot_features_cat_classification(df, target_col = "", columns = [], mi_threshold=0.0, normalize=False):
    
    # Comprobación de errores de entrada
    if not isinstance(df, pd.DataFrame):
        print("El argumento 'df' debe ser un DataFrame de pandas.")
        return None
    if target_col not in df.columns:
        print("El argumento 'target_col' debe ser una columna válida en el DataFrame.")
        return None
    if not isinstance(columns, list):
        print("El argumento 'columns' debe ser una lista de columnas.")
        return None
    if not all(col in df.columns for col in columns):
        print("Todas las columnas en 'columns' deben ser válidas en el DataFrame.")
        return None
    if not isinstance(mi_threshold, (int, float)):
        print("El argumento 'mi_threshold' debe ser un número.")
        return None
    if normalize:
        if not 0 <= mi_threshold <= 1:
            print("Cuando 'normalize' es True, 'mi_threshold' debe estar entre 0 y 1.")
            return None
    
    if not isinstance(normalize, bool):
        print("El argumento 'normalize' debe ser un valor booleano.")
        return None
    
    # si columns está vacío le metemos las columnas con una cardinalidad inferior a 20 que no sean target
    if not columns:
        columns = [col for col in df.columns if df[col].nunique() < 20 and col != target_col]
        
    # preparar la funcion de mutual info segun si se quiere normalizada o no
    if not normalize:
        mutual_info = mutual_info_score
    else:
        mutual_info = normalized_mutual_info_score

    # obtener mutual info y meter las features q superan el umbral en la lista para pintar
    para_pintar = []
    for col in columns:
        mis = mutual_info(df[target_col], df[col])
        if mis >= mi_threshold:
            para_pintar.append(col)

    # momento pintar
    num_cols = len(para_pintar)
    num_subplots_per_row = 2
    num_rows = (num_cols + num_subplots_per_row - 1) // num_subplots_per_row

    fig, axes = plt.subplots(num_rows, num_subplots_per_row, figsize=(12, num_rows * 5))

    for i, col in enumerate(para_pintar):
        row = i // num_subplots_per_row
        col_index = i % num_subplots_per_row
        ax = axes[row, col_index]
        
        sns.countplot(x=col, hue='survived', data=df, ax=ax)
        ax.set_title(f'Distribución de {target_col} por {col}')
        ax.set_xlabel(col)

    for i in range(num_cols, num_rows * num_subplots_per_row):
        axes.flatten()[i].axis('off')

    plt.tight_layout()
    plt.show()

$$*$$